# Evaluation Technique Jimini

## Présentation de Jimini

Pour rappel, nous développons trois outils :
- Un outil d'analyse factuelle de documents;
- Un outil de recherche juridique;
- Un outil de rédaction de documents juridiques, type contrats.

## Information Retrieval

### Objectif

Les deux premiers outils nécessitent d'être capable de récupérer le contexte pertinent dans un ensemble de paragraphes donné - problématique connue sous le nom d'`information retrieval` ou de `document retrieval`. L'objectif de cette partie est donc de construire un moteur de recherche juridique. Nous disposons d'un corpus de documents juridiques, et nous souhaitons que l'utilisateur puisse poser une question en langage naturel, puis que le moteur de recherche lui renvoie les documents les plus pertinents.

### Dataset

Dans le cadre de ce notebook, nous nous intéressons à un jeu de données juridique francophone : [Belgian Statuatory Article Retrieval Dataset (BSARD)](https://huggingface.co/datasets/maastrichtlawtech/bsard), qui contient ~1000 questions posées par les utilisateurs d'une plateforme d'information juridique belge (Droits Quotidiens), ainsi que les articles pertinents pour y répondre. Ce jeu de données est issu de l'article [A Statutory Article Retrieval Dataset in French](https://arxiv.org/pdf/2108.11792.pdf) de l'Université de Maastricht.

Ainsi que détaillé sur la page HuggingFace, le dataset se présente sous la forme de deux fichiers CSV contenant les questions d'entraînement `questions_fr_train.csv` et de test `questions_fr_test.csv`, et structurés comme suit :
 - `id` : un attribut int32 correspondant à un numéro d'identifiant unique pour la question.
 - `question` : un attribut de type chaîne de caractères correspondant à la question.
 - `category` : un attribut de type chaîne de caractères correspondant au sujet général de la question.
 - `subcategory` : un attribut de type chaîne de caractères correspondant au sous-sujet de la question.
 - `extra_description` : un attribut de type chaîne de caractères correspondant aux tags de catégorisation supplémentaires de la question.
 - `article_ids` : un attribut de type chaîne de caractères contenant les ID d'articles pertinents pour la question, séparés par des virgules.

(À noter, ce dataset contient également un échantillon de ~113k questions synthétiques, ainsi que des exemples de *négatifs* durs, c'est-à-dire des questions avec des articles non-pertinents, dont vous pouvez librement vous servir pour entraîner votre modèle.)

Et d'un fichier `articles.csv` :
 - `id` : un attribut int32 correspondant à un numéro d'identifiant unique pour l'article.
 - `article` : un attribut de type chaîne de caractères correspondant à l'intégralité de l'article.
 - `code` : un attribut de type chaîne de caractères correspondant au code de loi auquel appartient l'article.
 - `article_no` : un attribut de type chaîne de caractères correspondant au numéro de l'article dans le code.
 - `description` : un attribut de type chaîne de caractères correspondant aux titres concaténés de l'article.
 - `law_type` : un attribut de type chaîne de caractères dont la valeur est soit "regional" soit "national".

### Évaluation

Ainsi que dans l'[article](https://arxiv.org/pdf/2108.11792.pdf), nous évaluerons la performance du modèle en utilisant les métriques R@100, R@200, R@500, MAP@100 et MRR@100.

Cependant, en conditions réelles, nous souhaitons que le moteur de recherche renvoie les documents les plus pertinents en premier. Nous utiliserons donc la métrique NDCG@100, qui prend en compte l'ordre des documents renvoyés, ainsi que les métriques précédentes, avec un $k$ plus petit (par exemple $k=1$, $k=5$ et $k=10$).

#### Pyterrier

On se sert ici du framework Pyterrier qui permet assez simplement de prototyper et évaluer un moteur de recherche, mais **libre au candidat d'utiliser l'approche qu'il préfère**.
Dans le code suivant, on se sert des notations de `PyTerrier`, qui fonctionne avec les clés suivantes :
- `qid` : `str` l'identifiant de la question
- `docno` : `str` l'identifiant de l'article
- `query` : `str` le texte de la question
- `rel_docnos` : `List[str]` la liste des documents pertinents pour la question
- `score`: `float` le score de pertinence du document pour la question
- `label`: `int` la pertinence du document pour la question (1 si pertinent, 0 sinon)
- `qrels` : `Dict[str, Dict[str, int]]` un dictionnaire contenant les documents pertinents pour chaque question


## Loader les données et Pyterrier l'evaluation

In [3]:
import torch
import pyterrier as pt
import pandas as pd
import os

In [1]:
import torch

In [7]:
from jnius import autoclass
System = autoclass('java.lang.System')
print(System.getProperty('java.version'))


22.0.2


In [2]:
if not pt.started():
    pt.init()


PyTerrier 0.10.0 has loaded Terrier 5.10 (built by craigm on 2024-08-22 17:33) and terrier-helper 0.0.8



In [5]:
from datasets import load_dataset

In [6]:
# Load the dataset
datasets = load_dataset("maastrichtlawtech/bsard", 'questions')
train, test = datasets['train'], datasets['test']

# Convert to pandas dataframes
df_train = pd.DataFrame(train)
df_test = pd.DataFrame(test)
print(df_train.columns)
#print(df_test.head())

Index(['id', 'category', 'subcategory', 'question', 'extra_description',
       'article_ids'],
      dtype='object')


In [7]:
# Retrieve the corpus of articles from the dataset
#!wget https://huggingface.co/datasets/maastrichtlawtech/bsard/resolve/main/articles.csv
df_articles = pd.read_csv('data/articles.csv')
print(df_articles.columns)

Index(['id', 'reference', 'article', 'law_type', 'code', 'book', 'part', 'act',
       'chapter', 'section', 'subsection', 'description'],
      dtype='object')


In [9]:
# Retrieve the corpus of articles from the dataset
#!wget https://huggingface.co/datasets/maastrichtlawtech/bsard/resolve/main/negatives/bm25_negatives_train.json
df_neg = pd.read_json('data/negatives_bm25_negatives_train.json')
print(df_neg.columns)

Index([1102,   91,  474,  836, 1079,  850,  176,  342, 1044,  946,
       ...
       1037,   31,  702,  433,  712,  308,  387,  940,  364,   51],
      dtype='int64', length=886)


In [12]:
print(df_neg.shape)
print(df_neg.head)

(10, 886)
<bound method NDFrame.head of     1102   91    474    836    1079   850    176    342    1044  946   ...  \
0   7026   5730   948    729  21112    992   1203   5516  16811  2514  ...   
1   7011   5731   947  12102  18724    901   2130   5517  16806  2522  ...   
2   6855   5747  1094    611  18199   2486   1765   1056  17160  2486  ...   
3   7008   5791  1112    616  21055   4381   1655   1058  16810  2519  ...   
4   7009  13774  1102    759  21087  17275   1819   1234  16808  2531  ...   
5  22176   5463  1123    837   3309   1003  20060   1303   7037   886  ...   
6   7010    673  1104  12273  18714  17280   2335   1150  12048  2529  ...   
7   7024  18222  1114  12065   9078   2864   1210  16028   7036  2477  ...   
8  22105    755  1120  12041   3310   2930   2272   1067  17157  2482  ...   
9   7067    734  4813  12042  18593  16076   1197   1156    637   884  ...   

    1037   31     702    433    712   308   387    940    364    51    
0  12048  18702   5421   5495

In [11]:
print(df_train.shape)

(886, 6)


In [6]:
# Ensure that the article IDs are strings, and rename the column to 'docno', because of PyTerrier's expectations
df_articles['docno'] = df_articles['id'].astype(str)
df_articles["article"] = df_articles["article"].str.replace("[^a-zA-Z0-9 À-ÿ]", " ", regex=True).str.strip().str.replace(" +", " ", regex=True)

# Define a function to prepare the query dataframe
def prepare_queries(df):
    queries = pd.DataFrame({
        'qid': df['id'].astype(str),
        'query': df['question'].str.replace("[^a-zA-Z0-9 À-ÿ]", " ", regex=True).str.strip().str.replace(" +", " ", regex=True),
        'rel_docnos': [str(_id) for _id in df['article_ids']]
    })
    return queries

# Prepare the train and test query dataframes
train_queries = prepare_queries(df_train)
test_queries = prepare_queries(df_test)

# Convert relevance judgements to a DataFrame, for PyTerrier
#qrels = pd.DataFrame([
#    {'qid': str(qid), 'docno': str(docno), 'label': 1}
#    for qid, docnos in test_queries[['qid', 'rel_docnos']].itertuples(index=False)
#    for docno in ast.literal_eval(docnos)  # Convert the string representation of list back to a list
#])


In [7]:
print(test_queries.head())


    qid                                              query  \
0   775    Quels sont les critères communaux d insalubrité   
1  1053  A t on droit à l allocation de naissance en ca...   
2    72  Quels frais peut on ajouter lors d un recouvre...   
3   861       Comment se déroule une expulsion à Bruxelles   
4   786            Peut on m expulser en hiver en Wallonie   

                                 rel_docnos  
0                                     12024  
1                                   947,948  
2                            2041,2119,2138  
3                  5562,5563,5564,5565,5566  
4  5561,5562,5563,5564,5565,5566,5567,12124  


In [9]:
qrels = pd.DataFrame([
    {'qid': str(qid), 'docno': str(docno), 'label': 1}
    for qid, docnos in test_queries[['qid', 'rel_docnos']].itertuples(index=False)
    for docno in docnos
])


In [10]:
print(qrels.head)

<bound method NDFrame.head of       qid docno  label
0     775     1      1
1     775     2      1
2     775     0      1
3     775     2      1
4     775     4      1
...   ...   ...    ...
5080  512     ,      1
5081  512     6      1
5082  512     5      1
5083  512     9      1
5084  512     8      1

[5085 rows x 3 columns]>


On indexe les articles dans l'index PyTerrier (`DFIndex` avait un bug, donc on passe par un `IterDictIndex`)

In [11]:
# Indexing the articles using IterDictIndexer
if os.path.exists("./bsard_index"):
    shutil.rmtree("./bsard_index")
indexer = pt.IterDictIndexer("./bsard_index", overwrite=True)
docs = [{"docno": str(row["id"]), "text": row["article"]} for _, row in df_articles.iterrows()]
indexref = indexer.index(docs, fields=["text"])

index = pt.IndexFactory.of(indexref)


21:55:18.761 [main] ERROR org.terrier.structures.Index -- Cannot create new index: path C:\Users\Gaspard.BEAUDOUIN\Jemi\.\var\./bsard_index does not exist, or cannot be written to


JavaException: JVM exception occurred: Cannot create new index: path C:\Users\Gaspard.BEAUDOUIN\Jemi\.\var\./bsard_index does not exist, or cannot be written to java.lang.IllegalArgumentException

In [17]:
import shutil

In [19]:
# Define the index path
index_path = "./bsard_index"  # Simplify the path

# Remove the old index if it exists
if os.path.exists(index_path):
    shutil.rmtree(index_path)

# Ensure parent directory exists
os.makedirs(os.path.dirname(index_path), exist_ok=True)

# Initialize the indexer
indexer = pt.IterDictIndexer(index_path, overwrite=True)

# Prepare documents for indexing
docs = [{"docno": str(row["id"]), "text": row["article"]} for _, row in df_articles.iterrows()]

# Index the documents
indexref = indexer.index(docs, fields=["text"])

# Load the index
index = pt.IndexFactory.of(indexref)

22:05:27.478 [main] ERROR org.terrier.structures.Index -- Cannot create new index: path C:\Users\Gaspard.BEAUDOUIN\Jemi\.\var\./bsard_index does not exist, or cannot be written to


JavaException: JVM exception occurred: Cannot create new index: path C:\Users\Gaspard.BEAUDOUIN\Jemi\.\var\./bsard_index does not exist, or cannot be written to java.lang.IllegalArgumentException

Le `retriever` doit être capable, étant donné une question, de classer les articles par pertinence. Le format de sortie est un CSV avec trois colonnes :
- `qid` : `str` l'identifiant de la question
- `docno` : `str` l'identifiant de l'article
- `score` : `float` le score de pertinence de l'article pour la question

Libre au candidat de choisir l'approche qu'il préfère pour construire le `retriever`, l'essentiel étant qu'il construise un fichier CSV avec les colonnes `qid`, `docno` et `score`.

**Remarque** : Il peut ainsi y avoir plusieurs lignes pour une même question, si plusieurs articles sont pertinents (ce qui est souvent le cas)

In [12]:
# Define the retrieval model : for example dummy BM25 and TF-IDF
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF", properties={"c": 1.0})
bm25 = pt.BatchRetrieve(index, wmodel="BM25",  properties={"c": 1.0, "bm25.k_1": 1, "bm25.b": 0.6})

# Run the TF-IDF model and save the results
tfidf_results = tfidf.transform(test_queries)
tfid_results = tfidf_results[["qid", "docno", "score"]]
tfidf_results.to_csv("tfidf_results.csv", index=False)

# Run the BM25 model and save the results
bm25_results = bm25.transform(test_queries)
bm25_results = bm25_results[["qid", "docno", "score"]]
bm25_results.to_csv("bm25_results.csv", index=False)


NameError: name 'index' is not defined

In [ ]:
# Define the evaluation metrics
eval_metrics = ["recall_1", "recall_5", "recall_10", RR@10, AP@10, "recall_100", "recall_200", "recall_500", "map", "ndcg", RR@100, nDCG@100, AP@100]

# Load the results
tfidf_results = pd.read_csv("tfidf_results.csv")
bm25_results = pd.read_csv("bm25_results.csv")

# Evaluate the models
result = pt.Experiment(
    [tfidf_results, bm25_results],
    test_queries,
    qrels,
    eval_metrics,
    names=['TF-IDF', "BM25"],
)

print(result)


     name  recall_1  recall_5  recall_10     RR@10     AP@10  recall_100  \
0  TF-IDF       0.0       0.0   0.000751  0.000751  0.000125    0.010103   
1    BM25       0.0       0.0   0.000000  0.000000  0.000000    0.006042   

   recall_200  recall_500       map      ndcg    RR@100  nDCG@100    AP@100  
0    0.018233    0.041715  0.000445  0.012226  0.001147  0.002751  0.000262  
1    0.014570    0.035043  0.000343  0.013084  0.000790  0.001952  0.000144  


## Embeddings des articles et des questions d'entrainements

In [6]:
from transformers import CamembertTokenizer, CamembertModel

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
model = CamembertModel.from_pretrained('camembert-base')



C:\Users\Gaspard.BEAUDOUIN\Jemi\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# Pour utilisation sur un GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [8]:
torch.cuda.is_available()

True

In [15]:
from tqdm import tqdm
import numpy as np

In [19]:
# Chargement du fichier CSV
def load_data(file_path):
    df = pd.read_csv(file_path)
    df['subsection'] = df['subsection'].fillna('') 
    df['description']=df['description'].fillna('')
    df['article']=df['article'].fillna('')# Remplir les valeurs manquantes par une chaîne vide
    df['combined_text'] = df['article'] + ' ' + df['description'] + ' ' + df['subsection']
    return df

# Fonction pour obtenir l'embedding d'un texte
def get_embedding(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Moyenne des embeddings de tous les tokens pour obtenir une représentation de la séquence entière
    embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embedding.flatten()

# Calcul des embeddings pour tout le DataFrame avec suivi de la progression
def compute_embeddings(df, tokenizer, model, device):
    embeddings = []
    for text in tqdm(df['combined_text'], desc="Calcul des embeddings"):
        embedding = get_embedding(text, tokenizer, model, device)
        embeddings.append(embedding)
    df['embedding'] = embeddings
    return df

# Sauvegarde des embeddings dans un nouveau DataFrame et un fichier CSV dans le répertoire de travail de Kaggle
def save_embeddings_to_csv(df, output_path):
    # Créer un nouveau DataFrame avec l'ID des articles et leurs embeddings
    new_df = pd.DataFrame({
        'id': df['id'],
        'embedding': df['embedding'].apply(lambda x: ','.join(map(str, x.tolist())))
    })
    
    # Sauvegarder le nouveau DataFrame dans un fichier CSV dans le répertoire de travail de Kaggle
    new_df.to_csv(output_path, index=False)

    # Retourner le nouveau DataFrame avec les embeddings
    return new_df

# Pipeline principal
def main(file_path, output_path_csv):
    # Charger les données
    df = load_data(file_path)
    
    # Calculer les embeddings
    df = compute_embeddings(df, tokenizer, model, device)
    
    # Sauvegarder le nouveau DataFrame avec uniquement 'id' et 'embedding' dans un fichier CSV
    new_df_with_embeddings = save_embeddings_to_csv(df, output_path_csv)
    
    # Confirmation de la sauvegarde
    print(f"Fichier CSV sauvegardé dans : {output_path_csv}")
    
    return new_df_with_embeddings

In [20]:
file_path = 'data/articles.csv'  # Chemin vers votre fichier CSV d'articles
output_path_csv = 'data/embeddings.csv'  # Chemin de sortie pour le fichier CSV des embeddings

# Appeler la fonction principale pour calculer et sauvegarder les embeddings
new_df_with_embeddings = main(file_path, output_path_csv)

Calcul des embeddings: 100%|██████████| 22633/22633 [05:16<00:00, 71.62it/s] 


Fichier CSV sauvegardé dans : embeddings.csv


In [ ]:
print(embeddings.shape)
print(df_articles.shape)

torch.Size([21018, 1, 768])
(22633, 13)


#### Nouvelle tentative de l'embedding des articles

In [21]:
from sklearn.metrics.pairwise import cosine_similarity


Approche naive avec la similarité cosinus

In [29]:
# Générer des embeddings pour quelques questions et articles
question_embeddings = get_embedding(df_test['question'].iloc[3], tokenizer, model, device)
print(type(question_embeddings))


<class 'numpy.ndarray'>


In [30]:
# Charger les embeddings des articles depuis le fichier CSV
articles_embeddings_df = pd.read_csv('data/embeddings.csv')

# Convertir les embeddings sous forme de numpy array
articles_embeddings_df['embedding'] = articles_embeddings_df['embedding'].apply(
    lambda x: np.fromstring(x[1:-1], sep=',')
)
articles_embeddings = np.vstack(articles_embeddings_df['embedding'].values)

# Fonction pour obtenir l'embedding d'une question
def get_embedding(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Moyenne des embeddings de tous les tokens pour obtenir une représentation de la séquence entière
    embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embedding.flatten()

# Calculer la similarité cosinus
def find_top_k_nearest_neighbors(articles_embeddings, question_embedding, k=5):
    # Calculer la similarité cosinus entre les embeddings des articles et l'embedding de la question
    similarities = cosine_similarity(articles_embeddings, question_embedding.reshape(1, -1)).flatten()

    # Trouver les indices des k articles les plus proches
    top_k_indices = similarities.argsort()[-k:][::-1]

    return top_k_indices, similarities[top_k_indices]

# Pipeline pour trouver les articles les plus proches
def find_closest_articles(articles_embeddings_df, question, tokenizer, model, device, k=5):
    # Calculer l'embedding pour la question spécifique
    question_embedding = get_embedding(question, tokenizer, model, device)

    # Trouver les 5 articles les plus proches de la question
    top_k_indices, similarities = find_top_k_nearest_neighbors(articles_embeddings, question_embedding, k)

    # Extraire les articles les plus proches
    closest_articles = articles_embeddings_df.iloc[top_k_indices]

    return closest_articles, similarities


In [39]:
question = df_test['question'].iloc[10]

# Trouver les 5 articles les plus proches de cette question
closest_articles, similarities = find_closest_articles(articles_embeddings_df, question, tokenizer, model, device, k=10)

# Afficher les articles les plus proches avec leurs similarités
for i, (similarity, article) in enumerate(zip(similarities, closest_articles.itertuples()), 1):
    print(f"Article {i}: ID = {article.id}, Similarité = {similarity}")



Article 1: ID = 6761, Similarité = 0.919247901905167
Article 2: ID = 562, Similarité = 0.9176364318344205
Article 3: ID = 6774, Similarité = 0.9149302343814337
Article 4: ID = 3097, Similarité = 0.9118783561792776
Article 5: ID = 3117, Similarité = 0.9115224428066805
Article 6: ID = 6773, Similarité = 0.9103698353468617
Article 7: ID = 15493, Similarité = 0.9102208099508795
Article 8: ID = 5631, Similarité = 0.9101202592654821
Article 9: ID = 7216, Similarité = 0.9094317062420386
Article 10: ID = 5760, Similarité = 0.9088537012664306


In [40]:
print(df_test.iloc[10]['article_ids'])

6163,6164,6165,6928,6935,6936


In [ ]:
# Afficher les articles les plus proches avec leurs similarités
for i, (similarity, article) in enumerate(zip(similarities, closest_articles.itertuples()), 1):
    print(f"Article {i}: ID = {article.id}, Similarité = {similarity}")


In [ ]:
# Calculer la similarité cosinus
def find_top_k_nearest_neighbors(articles_embeddings, question_embedding, k=5):
    # Calculer la similarité cosinus entre les embeddings des articles et l'embedding de la question
    similarities = cosine_similarity(articles_embeddings, question_embedding.reshape(1, -1)).flatten()

    # Trouver les indices des k articles les plus proches
    top_k_indices = similarities.argsort()[-k:][::-1]

    return top_k_indices, similarities[top_k_indices]

# Pipeline pour trouver les articles les plus proches
def find_closest_articles(df_articles, df_test, tokenizer, model, device, k=5):
    # Embedding pour la question spécifique
    question_embedding = get_embedding(df_test['question'].iloc[0], tokenizer, model, device)  # Embedding de la question

    # Extraire les embeddings des articles déjà calculés
    articles_embeddings = np.vstack(df_articles['embedding'].values)

    # Trouver les 5 articles les plus proches de la question
    top_k_indices, similarities = find_top_k_nearest_neighbors(articles_embeddings, question_embedding, k)

    # Extraire les articles les plus proches
    closest_articles = df_articles.iloc[top_k_indices]

    return closest_articles, similarities

In [ ]:
file_path_articles = 'data/articles.csv'  # Chemin vers le fichier CSV d'articles
file_path_test = 'data/test.csv'  # Chemin vers le fichier CSV contenant la question dans df_test

# Charger les articles et calculer les embeddings
df_articles = load_data(file_path_articles)
df_articles = compute_embeddings(df_articles, tokenizer, model, device)

# Charger le fichier de test avec la question
df_test = pd.read_csv(file_path_test)

# Trouver les 5 articles les plus proches de la question
closest_articles, similarities = find_closest_articles(df_articles, df_test, tokenizer, model, device, k=5)

# Afficher les articles les plus proches avec leurs similarités
for i, (index, similarity) in enumerate(zip(closest_articles.index, similarities)):
    print(f"Article {i+1}: ID = {df_articles.iloc[index]['id']}, Similarité = {similarity}")

In [ ]:

# Supprimez la dimension inutile de taille 1 pour les deux ensembles d'embeddings
#question_embedding_np = question_embeddings.squeeze().numpy()
#article_embeddings_np = articles_embeddings.squeeze().numpy()

# Maintenant, calculez la similarité cosinus comme prévu
similarities = cosine_similarity(question_embedding_np.reshape(1, -1), article_embeddings_np)

# Le reste de votre code pour récupérer les top 5 articles peut rester le même
top5_idx = np.argsort(similarities[0])[-5:]
top5_articles_ids = df_articles.iloc[top5_idx]['id'].values

print("Top 5 des articles les plus similaires :", [top5_articles_ids[i]+5400 for i in range(len(top5_articles_ids))])
# devrait etre 5562

In [ ]:
print(all_train_embeddings.shape)
print(df_train.shape)


final_embeddings_path = os.path.join(save_dir, 'all_train_embeddings.pt')
torch.save(all_train_embeddings, final_embeddings_path)
print(f'Tous les embeddings ont été regroupés et sauvegardés dans {final_embeddings_path}')



torch.Size([886, 768])
(886, 6)
Tous les embeddings ont été regroupés et sauvegardés dans /content/drive/My Drive/Jimini/article_embeddings/all_train_embeddings.pt


## Chargement des embeddings et données

In [ ]:
all_train_embeddings=torch.load('/content/drive/My Drive/Jimini/article_embeddings/all_train_embeddings.pt' )
all_article_embeddings=torch.load('/content/drive/My Drive/Jimini/article_embeddings/all_article_embeddings.pt' )

<ipython-input-55-62ca7870e63b>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  all_train_embeddings=torch.load('/content/drive/My Drive/Jimini/article_embeddings/all_train

In [ ]:
import json

# Remplacez 'path_to_your_file.json' par le chemin réel de votre fichier JSON
#with open('negatives_bm25_negatives_train.json', 'r') as f:
#    negative_map = json.load(f)

negative_map = {int(k): v for k, v in df_neg.items()}

positive_map = {int(row['id']): [int(article_id.strip()) for article_id in str(row['article_ids']).split(',')] for _, row in df_train.iterrows()}



In [ ]:
print(all_article_embeddings.shape)
print(df_articles.shape)

torch.Size([21018, 1, 768])
(22633, 13)


## Optimisation et entrainement

In [ ]:
import random
from torch.utils.data import Dataset, DataLoader

class QuestionArticleDataset(Dataset):
    def __init__(self, questions, articles, positive_map, negative_map):
        self.questions = questions
        self.articles = articles
        # S'assurer que seules les clés présentes à la fois dans positive_map et negative_map sont utilisées
        self.keys = [key for key in positive_map.keys() if key in negative_map]
        self.positive_map = positive_map
        self.negative_map = negative_map

    def __len__(self):
        # Utiliser la longueur de self.keys pour déterminer le nombre d'éléments dans le Dataset
        return len(self.keys)

    def __getitem__(self, idx):
        # Utiliser self.keys[idx] pour obtenir l'ID de la question actuelle
        question_id = self.keys[idx]

        # Utiliser question_id pour accéder à positive_map et negative_map
        q_embedding = self.questions[idx]
        a_pos_index = random.choice(self.positive_map[question_id])  # Choix aléatoire d'un article positif
        a_neg_index = random.choice(self.negative_map[question_id])  # Choix aléatoire d'un article négatif

        a_pos_embedding = self.articles[a_pos_index]
        a_neg_embedding = self.articles[a_neg_index]

        return q_embedding, a_pos_embedding, a_neg_embedding

questions_embeddings = all_train_embeddings
articles_embeddings = all_article_embeddings
positive_map = positive_map
negative_map = negative_map

# Initialiser le Dataset
dataset = QuestionArticleDataset(questions_embeddings, articles_embeddings, positive_map, negative_map)

# Créer un DataLoader
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:

import torch.nn as nn
import torch.optim as optim

class Projector(nn.Module):
    def __init__(self, input_dim, proj_dim):
        super(Projector, self).__init__()
        self.fc = nn.Linear(input_dim, proj_dim)

    def forward(self, x):
        return self.fc(x)

# Initialisation des projecteurs
input_dim = 768  # taille de sortie d'un BERT
proj_dim = 128 # Dimension souhaitée des embeddings projetés
P_q = Projector(input_dim, proj_dim)
P_a = Projector(input_dim, proj_dim)
num_epochs=10
# Fonction de perte et optimiseur
margin = 1
loss_fn = nn.TripletMarginLoss(margin=margin, p=2) # margin garantit que les exemples similaires sont dans une marge de sécurité
optimizer = optim.Adam(list(P_q.parameters()) + list(P_a.parameters()), lr=0.001)

# entrainement
for epoch in range(num_epochs):
    epoch_loss = 0
    for q, a_pos, a_neg in data_loader:
        optimizer.zero_grad()
        q_proj = P_q(q)
        a_pos_proj = P_a(a_pos)
        a_neg_proj = P_a(a_neg)
        loss = loss_fn(q_proj, a_pos_proj, a_neg_proj)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(data_loader)}')


Epoch 1, Loss: 0.8497014088290078
Epoch 2, Loss: 0.6886797706995692
Epoch 3, Loss: 0.6008915188057082
Epoch 4, Loss: 0.5715664880616325
Epoch 5, Loss: 0.5482889626707349
Epoch 6, Loss: 0.513806477189064
Epoch 7, Loss: 0.4958223540868078
Epoch 8, Loss: 0.457973803792681
Epoch 9, Loss: 0.4020880439451763
Epoch 10, Loss: 0.3865130660789354


In [ ]:
save_path = '/content/drive/My Drive/Jimini/article_embeddings/P_q_projector_ex.pth'

# Enregistrement du modèle
torch.save(P_q.state_dict(), save_path)

In [ ]:
P_a.eval()

# Appliquez P_a à tous les embeddings
with torch.no_grad():
    projected_articles = P_a(all_article_embeddings)

# Chemin pour sauvegarder les embeddings projetés sur Google Drive
projected_embeddings_path = os.path.join('/content/drive/My Drive/Jimini/article_embeddings', 'all_projected_article_embeddings_ex.pt')

# Sauvegarder sur Google Drive
torch.save(projected_articles, projected_embeddings_path)
print(f'Les embeddings projetés ont été sauvegardés dans {projected_embeddings_path}')


Les embeddings projetés ont été sauvegardés dans /content/drive/My Drive/Jimini/article_embeddings/all_projected_article_embeddings_ex.pt


## Trouver l'article correpondant

charger les articles projettés et le projecteur de question

In [ ]:
# Création d'une nouvelle instance de projecteur
P_q_loaded = Projector(input_dim, proj_dim)

# Chargement des paramètres enregistrés
P_q_loaded.load_state_dict(torch.load('/content/drive/My Drive/Jimini/article_embeddings/P_q_projector_ex.pth'))

# Assurez-vous de passer le modèle en mode évaluation si vous l'utilisez pour des inférences
P_q_loaded.eval()

projected_articles=torch.load('/content/drive/My Drive/Jimini/article_embeddings/all_projected_article_embeddings_ex.pt')
print(projected_articles.shape)

Projector(
  (fc): Linear(in_features=768, out_features=128, bias=True)
)

In [ ]:
# Pour une question, trouver le plus proche voisin
from scipy.spatial import distance
import numpy as np



def find_closest_articles(question_text, description, projected_articles, tokenizer, model, projector, n=10):
    # Concaténation de la description avec la question
    combined_text = description + " " + question_text

    # Tokenizer le texte combiné
    inputs = tokenizer(combined_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Obtention des sorties du modèle
    with torch.no_grad():
        outputs = model(**inputs)

    # Utilisation des embeddings du dernier état caché et projeter
    embeddings = outputs.last_hidden_state.mean(dim=1)
    projected_q = projector(embeddings)

    # réduction des dimensions
    projected_q = projected_q.detach().squeeze()  # Réduire à un vecteur 1-D

    # Calculer les distances
    distances = []
    for i, projected_a in enumerate(projected_articles):
        projected_a = projected_a.squeeze()  # vecteur 1-D
        dist = distance.euclidean(projected_q.numpy(), projected_a.numpy())
        distances.append((dist, i))

    # Trouver les 10 plus petites distances
    distances.sort(key=lambda x: x[0])
    closest_ids = [distances[i][1] for i in range(n+5)]
    closest_dist=[[distances[i][0] for i in range(n+5)]]
    return closest_ids, closest_dist




# Trouver les IDs des articles les plus proches

print ("TRAIN")
print("\n")
closest_article_ids = find_closest_articles("Je suis travailleur salarié(e). Puis-je refuser de faire des heures supplémentaires ou de travailler de nuit ?", "Travail et parentalité, pendant la grossese", projected_articles, tokenizer, model, P_q)
print("Les IDs des 5 articles les plus proches sont :", closest_article_ids[0])
print("devrait être : [22225,22226,22227,22228,22229,22230,22231,22232,22233,22234]")
print("\n")
closest_article_ids = find_closest_articles("Je suis marié(e). Nous sommes mariés. Dois-je reconnaître mon enfant ?", "Situation de couples; Mariage", projected_articles, tokenizer, model, P_q)
print("Les IDs des 5 articles les plus proches sont :", closest_article_ids[0])
print("devrait être : [1096,1097,1098,1108,1109,1110]")
print("\n")
closest_article_ids = find_closest_articles("Qu'est-ce que le trajet de réintégration ?", "Santé et maladie, protection sociale, Reprise d'un travail adapté pendant l'incapacité, Trajet de réintégration", projected_articles, tokenizer, model, P_q)
print("Les IDs des 5 articles les plus proches sont :", closest_article_ids[0])
print("devrait être : [21114,21115,21116,21117,21118,21119,21120,21121,21122,21123,21124]")
print("\n")
print("\n")


print("TEST")
print("\n")
closest_article_ids = find_closest_articles("Peut-on me confisquer ou saisir ma voiture si on y trouve du cannabis ?", "Argent, dette, Procédures de récupération des dettes, Récupération amiable des dettes", projected_articles, tokenizer, model, P_q)
print("Les IDs des 5 articles les plus proches sont :", closest_article_ids[0])
print("devrait être : [2041,2119,2138]")

print("\n")
closest_article_ids = find_closest_articles("Quels frais peut-on ajouter lors d'un recouvrement amiable ?", "justice, Détenir, acheter et vendre du cannabis", projected_articles, tokenizer, model, P_q)
print("Les IDs des 5 articles les plus proches sont :", closest_article_ids[0])
print("devrait être : [6080,6120,6121,6122,6123,6124,6835,6769]")

for i in range(10):
  print("\n")
  closest_article_ids = find_closest_articles(df_train.iloc[i]['question'], df_train.iloc[i]['category']+df_train.iloc[i]['subcategory'], projected_articles, tokenizer, model, P_q)
  print("Les IDs des 5 articles les plus proches sont :", closest_article_ids[0])
  print("devrait être : ", df_test.iloc[i]["article_ids"])



TRAIN


Les IDs des 5 articles les plus proches sont : [2821, 907, 949, 6721, 22231, 6646, 2826, 1659, 952, 6734, 1153, 6383, 2822, 5468, 934]
devrait être : [22225,22226,22227,22228,22229,22230,22231,22232,22233,22234]


Les IDs des 5 articles les plus proches sont : [1017, 1045, 1084, 1091, 1089, 1080, 1061, 1053, 1018, 1030, 1090, 2447, 2360, 1068, 1019]
devrait être : [1096,1097,1098,1108,1109,1110]


Les IDs des 5 articles les plus proches sont : [823, 1150, 1306, 13912, 13404, 1113, 1305, 6507, 5572, 914, 5423, 1115, 1116, 1502, 1310]
devrait être : [21114,21115,21116,21117,21118,21119,21120,21121,21122,21123,21124]




TEST


Les IDs des 5 articles les plus proches sont : [5308, 2421, 2203, 5491, 2005, 981, 2205, 2804, 2805, 2157, 2787, 1065, 2002, 2162, 2029]
devrait être : [2041,2119,2138]


Les IDs des 5 articles les plus proches sont : [4825, 5162, 5557, 13658, 4821, 5947, 5181, 13914, 4961, 5216, 4822, 4843, 2976, 4766, 6502]
devrait être : [6080,6120,6121,6122,6123,6124,68

## Evaluation du retriever

In [ ]:
# inutile donne 0 ici
def give_score(articles1,articles2):
    score=0
    for article in articles1:
      if article in articles2:
        score+=1
    return score/len(articles2)


In [ ]:
print(test_queries.head)

<bound method NDFrame.head of       qid                                              query  \
0     775    Quels sont les critères communaux d insalubrité   
1    1053  A t on droit à l allocation de naissance en ca...   
2      72  Quels frais peut on ajouter lors d un recouvre...   
3     861       Comment se déroule une expulsion à Bruxelles   
4     786            Peut on m expulser en hiver en Wallonie   
..    ...                                                ...   
217   524  Je deviens cohabitant légal Dois je payer les ...   
218    69                       A qui dois je payer ma dette   
219   463  Je suis marié e On prend un logement en locati...   
220   880  Est ce que je peux signer plusieurs baux de co...   
221   512  Je suis victime de violences conjugales En tan...   

                                     rel_docnos  
0                                       [12024]  
1                                     [947,948]  
2                              [2041,2119,2138]  
3

In [ ]:
results = []

for idx, row in test_queries.iterrows():
    qid = row['qid']
    query_text = row['query']
    n=len(row['rel_docnos'])
    #description = df_test.iloc[idx]["category"]+df_test.iloc[idx]["subcategory"]+ df_test.iloc[idx]["extra_description"] # Ou utilisez une colonne de 'test_queries' si applicable
    description = df_test.iloc[idx]["category"]+df_test.iloc[idx]["subcategory"]
    closest_article_ids = find_closest_articles(query_text, description, projected_articles, tokenizer, model, P_q, n)[0]
    score=give_score(list(closest_article_ids),row['rel_docnos'])
    # Pour cet exemple, les scores sont définis comme inverses des rangs (simple exemple)
    for rank, article_id in enumerate(closest_article_ids):
        results.append({'qid': qid, 'docno': article_id, 'score': score})



In [ ]:
print(results_df)
file_path = '/content/drive/My Drive/Jimini/article_embeddings/custom_retrieval_results.csv'
# Enregistrer le fichier CSV
results_df.to_csv(file_path, index=False)


      qid  docno  score
0     775    855    0.0
1     775    856    0.0
2     775   1338    0.0
3     775   6591    0.0
4     775   1502    0.0
...   ...    ...    ...
1327  512  22445    0.0
1328  512  22449    0.0
1329  512   1087    0.0
1330  512   2830    0.0
1331  512   1078    0.0

[1332 rows x 3 columns]


## Quelques questions ouvertes

### Information Retrieval

Dans le cas de *Jimini Analyzer*, on est dans un contexte un peu particulier, où l'information provient de documents structurés.
- L'approche précédente ne tient compte que du contenu des paragraphes, pas de leur place dans la structure du document. Comment pourrait-on améliorer cela ?

- Que pensez-vous du dataset utilisé ? Comment pourrait-on l'améliorer ?
plus de données


- Quelle stratégie adopter pour faire un résumé d'un long document ?


### Raisonnement juridique


Récupérer le contexte pertinent est une étape *sine qua none*, mais il faut ensuite fournir le contexte au LLM, et qu'il réponde à la question posée.
- Comment évaluer le LLM sur le contenu de ses réponses ?
- Quelles métriques utiliser ?
- Quel dataset utiliser / construire pour faire acquérir un raisonnement juridique au LLM ?
- Comment s'assurer que le dataset est de qualité / représentatif ?  
- De quel modèle partir ?
- Quelle technique d'entraînement utiliser ? (full-parameter fine-tuning, PEFT - LoRA - QLoRA , prompt-tuning,...)
- Comment contrôler l'hallucination ?
- Comment faire de l'amélioration continue, en s'assurant que le modèle ne se dégrade pas lorsqu'on lui enseigne une nouvelle compétence ?
- Comment limiter les frais de calculs ?


### Rédaction de documents

- Comment rédiger un contrat de plusieurs dizaines de pages ?

### Bonus

Sentez-vous libre de proposer des améliorations, des idées, des pistes, etc. !


